# DatasetとDataLoaderについて
pytorchには、データセットを扱う基本要素として以下の二つがある。  
1. `torch.utils.data.Dataset`  
2. `torch,utils.data.DataLoader`  

これらを組み合わせることで、データの使用を簡単にしてくれる。  

(1) Datasetについて  
> Datasetには、**サンプル**と。それに対応する**ラベル**が格納される  

(2) DataLoadeについて  
> Datasetを引数として渡すことで、イテレート」処理が可能なものへとラッピングする。  
> イテレート処理 : 100このデータセットを10個ずつに分けて渡してくれるイメージ  

## pytorchのDatasetについて
pytorchでは、多くのデータセットが用意されていて、主に以下の三つに分けられる。  
1. 画像データ : [画像データの詳細は](https://pytorch.org/docs/stable/torchvision/datasets.html)
2. テキストデータ : [テキストデータの詳細](https://pytorch.org/docs/stable/torchvision/datasets.html)
3. 音声データ : [音声データの詳細](https://pytorch.org/audio/stable/datasets.html)


# Datasetからの読み込み  
pytorchで用意されているDatasetを利用するには、主に、`TorchVision`を用いる。  
以下、TorchVisionからFASION-MNISTをロードする方法を紹介する。  
FASION-MNISTとは、60,000個の訓練データと10,000個のテストデータから構成された、Zalandoの記事画像のデータセットです。

各サンプルは、28×28のグレースケール画像と、10クラスのうちの1つのラベルから構成されています。

### TorchVisionを用いたデータのロード
TorchVisionを用いてデータのロードを行う際には、以下のパラメータを使用する。
1. `root` : train/testデータが格納されているパスを指定  
2. `train` : 訓練データまたがテストデータを指定  
3. `download=True` : rootにデータが村座しない場合、インターネットからダウンロードしてくる  
4. `transform` `target_transofrm` : 特徴量とラベルの変換を指定

In [ ]:
%matplotlib inline

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt


In [ ]:
# Datasetによるデータの読み込み

# trainデータの読み込み
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# testデータの読み込み
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


## データセットの反復処理と可視化
Datasetの特定のindexを指定するにはリスト操作と同様にすれば良い。`data[index]`  
matplotlibを用いていくつかのデータを可視化する

In [ ]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

figure = plt.figure(figsize=(8,8))
cols, rows = 3, 3
for i in range(1, cols*rows+1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

# カスタムデータセットの作成 (自分でデータせセットを作る)
自分でカスタムしたDatasetクラスを作成する際には、以下の3つの関数を**必ず**実装する必要がある。  
1. `__init__`  
2. `__len__`  
3. `__getitem__`  

以下では仮に、
FashionMNISTの画像データを`img_dir`フォルダに、  
ラベルはcsvファイルとして、`annotations_file`として保存してあるとする。

(1) `__init__`について
> `__init__`関数はDatasetオブジェクトがインスタンス化されるときに1度だけ実行される。  
> 画像、annotation_file, それらに対する変換処理の初期設定を行う。  

(2) `__len__`について  
> `__len__`関数は、データセットのサンプル数を返す関数  

(3) `__getitem__`について  
> `__getitem__`関数は、指定された`idx`に対応するサンプルをデータセットから読み込んで返す関数。  
> ここでは、`index`に基づいて、画像ファイルのパスを特定し、`read_image`を使用して画像をテンソルに変換する。  
> さらに、transform functionを必要に応じて適用し、最終的に辞書型でデータとラベルを返す。

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, \
                 transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        sample = {"image": image, "label":label}
        return sample

# DataLoaderの使い方
Datasetでは、indexを指定してデータを取り出すことができる。が、  
実際には、ミニバッチ単位で取り出すことがおおい。(具体的には、10こずつランダムに取り出すのように)  
また、過学習を防ぐために、**epochでデータをシャッフルする**  
これらの操作を簡単にするためのAPIとして`DataLoader`が存在する。

## DataLoaderを用いた反復処理
第一に、`DataLoader`を使用するために、`Dataset`を読み込ませる。  
その後、以下に示すようにして、各データとそれに対応するラベルを取り出すことができる。  
このとき、`shuffle=True`とすることで、順番をシャッフルすることができる。  
より詳細な情報は[データの読み込みの詳細](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler)を参照  

### さらなる詳細について
`torch..utils.data`の詳細は[torch.utils.dataの詳細](https://pytorch.org/docs/stable/data.html)を参照

In [ ]:
from torch.utils.data import DataLoader

# データセットを読み込ませる
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
train_feature, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape : {train_feature.size()}")
print(f"Labels batch shape : {train_labels.size()}")
img = train_feature[0].squeeze()
label = train_labels[0]
print(f"Label : {label}")
plt.imshow(img, cmap="gray")
plt.show()
